# Overview

PyLag is an offline particle tracking model. The model expects as inputs time independent and/or dependent variables that describe the state of a given fluid. These may be measured quantities or the predictions of an analytical or numerical model. Using these, the model computes Lagrangian trajectories for particles released into the fluid at a particular point in time and space. The model is primarily aimed at marine applications, but is intended to be versatile enough to be extended to other contexts; for example, studies of atmospheric dispersion.

PyLag was created with the aim to make available a particle tracking model that is a) fast to run, b) easy to use, c) extensible and d) flexible. The model is written in a mixture of [Python](http://www.python.org), [Cython](http://www.cython.org) and C++.

## Code structure

A simplified Unified Modelling Language (UML) Class Diagram for the model is shown in Figure 1. Functions that perform numerically intensive tasks have been implemented in Cython or C++ (dark grey boxes), while those that perform less numerically intensive tasks have been implemented in Python (pale grey boxes). The division is intended to combine the ease of use of Python with the efficiency gains offered by Cython and C++. During compilation, Cython source files are parsed and translated into optimized C++ code, which is them compiled into extension modules that can be loaded at runtime by the Python Interpreter.

![Code structure](figures/fig_code_structure_a.png)
Figure 1: UML Class Diagram.

The model currently includes direct support for the General Ocean Turbulence Model (GOTM), the Finite Volume Community Ocean Model (FVCOM) and data defined on a regular Arakawa A-grid (not shown in Figure 1). GOTM is a one-dimensional, relocatable water column model that includes state-of-the-art descriptions of vertical turbulent mixing (Umlauf et al 2008). FVCOM is a prognostic, unstructured-grid, finite-volume, free-surface, 3D primitive equation coastal ocean circulation model (Chen 2003). Arakawa A-grids are a common grid format, and it is often a requirement that data be provided on an Arakawa A-grid before being submitted to public data repositories such as the [CMEMS catalogue](https://marine.copernicus.eu/).

Each input source has associated with it a derived `DataReader` class that inherits from a base class `DataReader` (see Figure 1). The primary purpose of such objects is to compute and provide access to the value of a given variable (e.g. the vertical turbulent eddy diffusivity) at a particular point in space and time. For gridded input data, this process is facilitated through interpolation, while taking into account the structure of the underlying model grid. Figures 1B and 1C give examples of the types of input generated by GOTM and FVCOM respectively: the former shows a hovemoller diagram of the simulated vertical turbulent eddy diffusivity for heat at a location in the Western English Channel; the latter a map of the surface horizontal turbulent eddy diffusivity field near to the coastal city of Plymouth, UK. The code has been designed in a way that makes it possible to easily extend the model to read in data defined on different types of input grid. This can be achieved by sub-classing `DataReader`. 

When working with 2-D or 3-D data, PyLag creates an unstructured horiztonal grid or grids to assist with locating particles and interpolating input data to particle positions. This occurs irrespective of the type of grid on which the input data is defined. When working with FVCOM, which uses an unstructured grid, PyLag simply adopts FVCOM's grid directly, after ensuring all information regarding neighbouring elements is ordered as PyLag expects. When working with data defined on a regular Arakawa A-grid, PyLag first creates an unstructured representation of the grid. In all cases, the required information is encoded within a dedicated grid metrics file which PyLag reads at the start of the simulation.

Given input data, PyLag can compute particle trajectories using one of the different numerical schemes implemented in the model code. In order to support numerical methods that employ some form of operator splitting in which advection and diffusion are handled separately, two different families of object have been implemented in the code. Objects that belong to the first family control the integration process, and may or may not implement some form of operator splitting. They inherit their interface from the base class `NumMethod`. Two derived classes have been implemented: `StdNumMethod` and `OSONumMethod`. The former does not employ operator splitting, while the latter implements a basic form of operator splitting. Objects from the second family represent iterative methods that have no awareness of whether operator splitting is being used: they simply compute position deltas, and return these to the caller. They inherit their interface from the base class `ItMethod`. Several derived classes have been implemented: some of these implement pure deterministic solution methods that should be used for advection-only problems; others implement purely stochastic solution methods that should be used for diffusion-only problems; while others implement solution methods that are suitable for problems involving both diffusion and advection.

The testing of such models is facilitated through the inclusion of various mock classes that are derived from `DataReader`. Mock classes typically encode analytic functions that describe the spatio-temporal variability of a given field variable or variables (e.g. the velocity field). This removes the dependency on external input data. Figure 1A shows an example of a vertical turbulent eddy diffusivity profile computed using an analytic formula. The same profile is used in one of PyLag's [examples](../examples/vertical_mixing_analytic.ipynb) to test for the Well Mixed Condition.

## Supported sources of input data

PyLag currently includes direct support for the following types of input data:

* [General Ocean Turbulence Model (GOTM)](http://gotm.net/)
* [Finite Volume Communitty Ocean Model (FVCOM)](http://fvcom.smast.umassd.edu/fvcom/)
* Regularly gridded data on an Arakawa A-grid (Arakawa-A), such as those available from the [CMEMS catalogue](https://marine.copernicus.eu/)

Several examples of running PyLag in different modes and with different types of input data are described in the various [tutorial examples](../examples).

## MPI support

The particle tracking model lends itself to parallelization since the particle set can be readily broken up and scattered over multiple processors that independently compute changes in state for each particle they manage. This approach has been applied using [MPI for Python](https://mpi4py.readthedocs.io/en/stable/). A Mediator class (Figure 1) facilitates the switching between serial and parallel execution without the need to either recompile the code or unnecessarily set operating system environmental variables.

![MPI scaling](figures/fig_mpi_scaling.png)
Figure 2: MPI scaling.

Short duration benchmark runs using a seed of $10^6$ particles show that, when compared with serial execution, this can reduce run times by a factor of 50 or more; however, for large numbers of processors (100s or more), gains can become limited by broadcast and gather operations, as well as the time needed to import shared libraries (Figure 2). Ultimately, the performance of MPI runs will depend on the run being performed. Generally, performance will be greatest in longer runs involving many particles, and in which read/write operations are performed relatively infrequently.

## Required inputs

For typical use cases, PyLag requires three types of input:

* A [run configuration file](./configuration.ipynb), in which all run parameters and paths are set.
* NetCDF data files, such as those produced by a given hydrodynamic model. The path to these is set in the run configuration file.
* A grid metrics file, which describes the underlying grid on which input data are defined.

The run configuration file is described in more detail at the above link, while example NetCDF data files are provided with the [tutorial examples](../examples/index.rst). Any number of input data files can be provided. The primary requirement of these is that they are named using a pattern that allows them to be sorted in sequential order according the time slices they cover. For example, two data files for January and February 2002 may have the names `file_1.nc` and `file_2.nc`. Using appropriately constructed datestrings in the file name is usually sufficient to guarentee sequential ordering.

The grid metrics file is included as it saves on having to compute the same information at the start of each run. It provides all the information PyLag needs to correctly read in and interpret the gridded input data used with the run. Special functions are provided in the module `pylag.grid_metrics` to help create grid metrics files that are specific to the type of input data being used.


## Running PyLag

PyLag can be run in serial or parallel modes. After installing PyLag using conda and setting up the run, a simulation can be run in serial using the following command:

```bash

$ python -m pylag.main -c pylag.cfg

```

where `pylag.cfg` is the run configuration file.

To launch a parallel run, simply type:

```bash

$ mpiexec -np n_proc python -m pylag.parallel.main -c pylag.cfg

```

where `n_proc` is the number of processors the run will be distributed over.

<div class="alert alert-info">

**Note:**

For the time being, the number of processors used must be an exact divisor of the total number of particles. If this is not the case, an error will be raised and the simulation halted.

</div>

## Analysing simulation outputs

The accompanying PyLag-tools repository provides a number of plotting tools to assist with analysing simulation outputs. Examples of their usage can be found in the [tutorial examples](../examples/index.rst).


## References

Chen,  C.  H.  Liu,  R.  C.  Beardsley,  2003.  An  unstructured,  finite-volume,  three-dimensio nal, primitive  equation  ocean  model:  application  to  coastal  ocean  and  estuaries.J.  Atm.  &Oceanic Tech., 20, 159-186

Umlauf, L. Burchard, H., 2004. Second-order turbulence closure models for geophysical boundary layers. A review of recent work. Continental Shelf Research 25(7):795-827. DOI: 10.1016/j.csr.2004.08.004